# Convolutional Neural Networks using `word2vec`

In [6]:
import torch
from torchtext import data
from torchtext import datasets
import random
import nltk
import re 
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import seaborn as sns 
import matplotlib.pyplot as plt

In [7]:
SEED = 1984
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True # for reproducible results 

In [8]:
def clean_text(text):
    text = text.lower()
    text = re.compile('<.*?>').sub(" ", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    text = text.split()
    return text

TEXT = data.Field(tokenize=clean_text,lower=True, fix_length=800)
LABEL = data.LabelField(dtype = torch.float)

In [14]:
train_df = pd.read_csv('us_news_train.csv')
test_df = pd.read_csv('us_news_test.csv')
validation_df = pd.read_csv('us_news_validation.csv')

In [19]:
train_df.head()

,outlet,outlet_url,datetime,url_orig,headline,description,author,domain,topic_tags,text,section,news_keywords,subsection,paywall,provider,ideology
0,cnbc,https://www.cnbc.com/,2019-03-15T09:00:46Z,https://www.cnbc.com/2019/03/15/iea-report-ven...,Venezuela's electricity crisis could trigger '...,A nationwide power failure in crisis-stricken ...,Sam Meredith,Oil,"WTI Crude (Apr'19), ICE Brent Crude (May'19), ...",A nationwide power failure in crisis-stricken ...,NaN,"Venezuela, oil, electricity, IEA, report, OPEC...",NaN,NaN,NaN,center
1,cnbc,https://www.cnbc.com/,2018-10-01T10:30:36Z,https://www.cnbc.com/2018/10/01/canadas-husky-...,Canada's Husky Energy offers to buy MEG Energy...,Canadian oil and gas producer Husky Energy sai...,CNBC,Energy,"Oil and Gas, Business, Energy, Canada, US: New...",Canadian oil and gas producer Husky Energy sai...,NaN,"Husky Energy, MEG, mergers and acquisitions, M...",NaN,NaN,NaN,center
2,cnbc,https://www.cnbc.com/,2018-08-16T13:15:32Z,https://www.cnbc.com/2018/08/16/travel-apps-to...,3 innovative new travel apps that save time an...,From the cheapest rideshare to 30 percent off ...,Jimmy Im,Make It - Life,"Tourism, Travel, Career advice, Entrepreneursh...",Millennials are more likely to increase vacati...,NaN,"make it, hoteltonight, daily drop, jetlag, tim...",NaN,NaN,NaN,center
3,cnbc,https://www.cnbc.com/,2018-07-25T12:27:28Z,https://www.cnbc.com/2018/07/25/germanys-econo...,Germany’s economy is showing ‘signs of fear’ o...,Trade tensions between the U.S. and Europe cou...,Holly Ellyatt,Europe News,"U.S. Economy, Trade, Asia Economy, Europe News...",Trade tensions between the U.S. and Europe cou...,NaN,"Europe US trade, trade war, Germany trade war,...",NaN,NaN,NaN,center
4,thehill,https://www.thehill.com/,2018-11-15T11:00:12Z,https://thehill.com/homenews/administration/41...,The Memo: Is Michelle Obama the one critic Tru...,Former first lady Michelle Obama has slammed P...,Niall Stanage,Administration,"Hillary Clinton, Michelle Obama, Donald Trump,...",Former first lady Michelle Obama has slammed P...,Homenews,NaN,NaN,NaN,NaN,center


In [ ]:
# example code to figure out what's up 
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(text_field=TEXT,label_field=LABEL,)
train_data, valid_data = train_data.split(split_ratio= 0.8, random_state=random.seed(SEED))

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [01:03<00:00, 5.38MB/s]
